# Quality Assurance for the ETL process with the houses data

University: ICESI

Student: Nicolas Esteban Colmenares Ruiz

Subject: Infraestructura y arquitectura de TI

Professor: Monica Rojas y Angela Villota

## Imports required for validation

In [89]:
import psycopg2
import pandas as pd
import pandas.io.sql as sqlio
import numpy as np

## Load data from ElephantSQL

In [130]:

# Connect to the database
conn = psycopg2.connect(
        host="isilo.db.elephantsql.com",
        database="krenswwj",
        port="5432",
        user="krenswwj",
        password="VNjHXDgtnsdqvbJovcrPhfvcxAd0rqkr")
sql = "SELECT * FROM amesdbtemp"
amesdbtemp = sqlio.read_sql_query(sql, conn)
# select table floordetails
sql = "SELECT * FROM floordetail"
floordetails = sqlio.read_sql_query(sql, conn)
# select table saleProperty 
sql = "SELECT * FROM saleproperty"
saleproperty = sqlio.read_sql_query(sql, conn)
conn.close()

<ipython-input-130-8ccbae99b468>:9: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  amesdbtemp = sqlio.read_sql_query(sql, conn)
<ipython-input-130-8ccbae99b468>:12: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  floordetails = sqlio.read_sql_query(sql, conn)
<ipython-input-130-8ccbae99b468>:15: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  saleproperty = sqlio.read_sql_query(sql, conn)


### Check the columns of both tables

In [3]:
# print columns of the table amesdbtemp
print("amesdbtemp columns",data.columns)
# print columns of the table floordetail
print("floor detail columns", floordetails.columns)

amesdbtemp columns Index(['Exter Cond', 'Exter Qual', 'Heating QC', 'Kitchen Qual', 'MS Zoning',
       'pid', 'Roof Style', 'Roof Matl', 'Exterior 1st', 'Exterior 2nd',
       'Mas Vnr Type', 'Mas Vnr Area', 'foundation', 'heating', 'Central Air',
       'electrical', '1st Flr SF', '2nd Flr SF', 'Low Qual Fin SF',
       'Kitchen AbvGr', 'TotRms AbvGrd', 'functional', 'fireplaces',
       'Fireplace Qu', 'Paved Drive', 'Wood Deck SF', 'Open Porch SF',
       'Enclosed Porch', '3Ssn Porch', 'Screen Porch', 'fence', 'MS SubClass'],
      dtype='object')
floor detail columns Index(['pid', 'Floor', 'bedrooms', 'Full Bath', 'Half Bath'], dtype='object')


## Load data from csv output of the ETL process

In [4]:
# Load RetoOutput.csv
df = pd.read_csv('RetoOutput.csv', sep=';')

## Validate quality requirements of resulting data

### check nulls

In [5]:
# print amount of NaN values in df
print("NaN values in df:", df.isnull().sum().sum())

#print amount of 0 values in df
print("cero values in df:", (df == 0).sum().sum())

NaN values in df: 8201
cero values in df: 35498


Those correspond to the NA and 0 replaced values during the transformation of data.

### check if there is empty values wich means wrong process during ETL

In [129]:
print((df == "").sum().sum())

0


There are no empty values, which are the result in the output when there is a null value to be load in the end of the ETL process. Therefore, there is no remaining nulls values in the output file.

## Compare postgres data with csv

### Final number of rows must be equals to the rows in database

In [6]:
# print rows of RetoOutput.csv and amesdbtemp table
print("Cantidad de filas en el final",df.shape[0])
print("Cantidad de filas en la tabla",data.shape[0])

Cantidad de filas en el final 2930
Cantidad de filas en la tabla 2930


### Validate fullbath, halfbath and bedrooms operation is correct

In [7]:
# get floordetails table and group by pid
floordetails_group = floordetails.groupby('pid').agg({
    'bedrooms': 'sum',
    'Full Bath': 'sum',
    'Half Bath': 'sum',
    }).reset_index()
# sort by pid
floordetails_group = floordetails_group.sort_values(by=['pid'])
print(floordetails_group)

             pid  bedrooms  Full Bath  Half Bath
0      526301100         3          1          0
1      526302030         2          2          0
2      526302040         2          1          0
3      526302110         2          1          0
4      526302120         3          1          1
...          ...       ...        ...        ...
2925   924100070         2          1          0
2926   924151040         3          2          1
2927   924151050         3          2          1
2928   924152030         3          2          1
2929  1007100110         4          2          0

[2930 rows x 4 columns]


In [11]:
# print the df halfBath, fullbath, bedroom
#ordenar por pid
df = df.sort_values(by=['PID'])
df_floor_sum = df[['PID','Bedroom','FullBath','HalfBath']]
print(df_floor_sum)

             PID  Bedroom  FullBath  HalfBath
0      526301100      3.0       1.0       0.0
1      526302030      2.0       2.0       0.0
2      526302040      2.0       1.0       0.0
3      526302110      2.0       1.0       0.0
4      526302120      3.0       1.0       1.0
...          ...      ...       ...       ...
2925   924100070      2.0       1.0       0.0
2926   924151040      3.0       2.0       1.0
2927   924151050      3.0       2.0       1.0
2928   924152030      3.0       2.0       1.0
2929  1007100110      4.0       2.0       0.0

[2930 rows x 4 columns]


In [13]:
# convert df_floor_sum values to int
df_floor_sum = df_floor_sum.astype(int)
df_floor_sum

,PID,Bedroom,FullBath,HalfBath
0,526301100,3,1,0
1,526302030,2,2,0
2,526302040,2,1,0
3,526302110,2,1,0
4,526302120,3,1,1
...,...,...,...,...
2925,924100070,2,1,0
2926,924151040,3,2,1
2927,924151050,3,2,1
2928,924152030,3,2,1


In [15]:
# check floordetails_group and df_floor_sum BedRoom, FullBath, HalfBath values are equal
print("Bedroom values are equal:", (floordetails_group['bedrooms'] == df_floor_sum['Bedroom']).all())
print("FullBath values are equal:", (floordetails_group['Full Bath'] == df_floor_sum['FullBath']).all())
print("HalfBath values are equal:", (floordetails_group['Half Bath'] == df_floor_sum['HalfBath']).all())


Bedroom values are equal: True
FullBath values are equal: True
HalfBath values are equal: True


Bedroom, Fullbath and Halfbath are the expected

### Validate Gr Liv Area is well calculated

In [ ]:
amesdbtemp['1st Flr SF'].astype(int) + amesdbtemp['2nd Flr SF'].astype(int)  + amesdbtemp['Low Qual Fin SF'].astype(int) 

In [108]:
amesdbtemp['1st Flr SF']

1849    1656
2061    1293
317     1024
2360     882
1866    1797
        ... 
2082    1389
87      1803
1729     996
101     1152
1537    1150
Name: 1st Flr SF, Length: 2930, dtype: Int64

In [123]:
# sort data by PID
data = amesdbtemp.sort_values(by=['pid'])

# create field Gr Liv Area from amesdbtemp table suming 1st Flr SF, 2nd Flr SF and Low Qual Fin SF
data['Gr Liv Area'] =  data[['1st Flr SF','2nd Flr SF','Low Qual Fin SF']].sum(axis=1).astype(int)

# reset indexes
data = data.reset_index(drop=True)

In [66]:
# change df type grLivArea to int
df['Gr Liv Area'] = df['Gr Liv Area'].astype(str).str.replace(',', '.').astype(float).astype(int)

In [126]:
# check Gr Liv Area values are equal to df GrLivArea
print("Gr Liv Area values are equal:", (data['Gr Liv Area'].equals(df['Gr Liv Area'])))

Gr Liv Area values are equal: True


correct calculation of GR liv Area

## Validate Mo Sold and Yr Sold are well calculated from sale property table

In [ ]:
# sort sale property and create a date time index frame to get year, month and day of each date
saleproperty = saleproperty.sort_values(by=['pid'])
dates = pd.DatetimeIndex(saleproperty['Sale Date'])

In [141]:
# check year and month from dates are equal to df YrSold and MoSold
print("Year values are equal:", (dates.year == df['Yr Sold']).all())
print("Month values are equal:", (dates.month == df['Mo Sold']).all())

Year values are equal: True
Month values are equal: True


the year and month extracted in the ETL process are correct.